<a href="https://colab.research.google.com/github/StTronn/website_cat_api/blob/master/WebCrawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Filter Cisco Rank

In [0]:
!wget -P /content/drive/My\ Drive/ -c "http://s3-us-west-1.amazonaws.com/umbrella-static/top-1m-2020-01-01.csv.zip"
!unzip -q "/content/drive/My Drive/top-1m-2020-01-01.csv.zip" -d "/content/drive/My Drive/Site Data"

In [0]:
import csv
import sys

csv.field_size_limit(sys.maxsize)

with open("/content/drive/My Drive/Site Data/top-1m.csv") as csvfile:
  csv_reader = csv.reader(csvfile)
  urls = list(csv_reader)

In [0]:
!pip3 install tldextract

In [0]:
from tldextract import extract
from tqdm import tqdm

done = set()
domains, subdomains, rejected = [], [], []

for url in tqdm(urls):
  url[1] = 'https://' + url[1]
  ext = extract(url[1])
  domain = ext.domain
  subdomain = ext.subdomain
  suffix = ext.suffix
  if subdomain == "" or subdomain == "www":
    if domain not in done:
      domains.append(url)
      done.add(domain)
    else:
       rejected.append(url)
  else: subdomains.append(url)
  

100%|██████████| 1000000/1000000 [00:09<00:00, 108923.88it/s]


In [0]:
print("domains:", len(domains), "subdomains:", len(subdomains), "rejected:", len(rejected))   

domains: 216493 subdomains: 695988 rejected: 87519


In [0]:
print(domains[0:100])
print(subdomains[0:100])
print(rejected[0:100])

[['1', 'https://netflix.com'], ['9', 'https://google.com'], ['10', 'https://microsoft.com'], ['13', 'https://windowsupdate.com'], ['17', 'https://live.com'], ['20', 'https://facebook.com'], ['23', 'https://nflxso.net'], ['24', 'https://doubleclick.net'], ['30', 'https://nflximg.com'], ['35', 'https://skype.com'], ['36', 'https://apple.com'], ['41', 'https://google-analytics.com'], ['44', 'https://bing.com'], ['45', 'https://office365.com'], ['46', 'https://amazonaws.com'], ['47', 'https://youtube.com'], ['48', 'https://www.googleapis.com'], ['52', 'https://digicert.com'], ['53', 'https://googleusercontent.com'], ['58', 'https://yahoo.com'], ['60', 'https://akamaiedge.net'], ['62', 'https://microsoftonline.com'], ['65', 'https://fbcdn.net'], ['68', 'https://windows.net'], ['69', 'https://googlesyndication.com'], ['73', 'https://office.com'], ['74', 'https://msn.com'], ['76', 'https://ytimg.com'], ['77', 'https://icloud.com'], ['78', 'https://googleadservices.com'], ['82', 'https://googl

In [0]:
with open("/content/drive/My Drive/Site Data/domains.csv", 'w') as csvfile:
  csv_writer = csv.writer(csvfile)
  csv_writer.writerows(domains)

with open("/content/drive/My Drive/Site Data/subdomains.csv", 'w') as csvfile:
  csv_writer = csv.writer(csvfile)
  csv_writer.writerows(subdomains)

In [0]:
with open("/content/drive/My Drive/Site Data/domains.csv") as csvfile:
  csv_reader = csv.reader(csvfile)
  domains = list(csv_reader)

In [0]:
domains[:100]

# Preprocessing

In [0]:
import warnings

warnings.filterwarnings("ignore")

In [0]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [0]:
from gensim.models import KeyedVectors

EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz' 

model = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True, limit=1000000)

In [0]:
all_keys = set()
for key in model.wv.vocab.keys():
  all_keys.add(key)
len(all_keys)

1000000

In [0]:
# This will also remove most of the names from dictionary along with not alphabetic keys and smaller keys"""

dictionary = set()
for key in model.wv.vocab.keys():
  if(key.isalpha() and len(key) > 2 and key.lower() in all_keys): 
    dictionary.add(key.lower())
len(dictionary)

104808

In [0]:
import csv

with open('/content/drive/My Drive/dictionary.csv', 'w') as csvfile:
  csv_writer = csv.writer(csvfile)
  csv_writer.writerow(dictionary)

In [0]:
#common web terms
AVOID = {'page', 'policy', 'url', 'load', 'unsupported', 'uses', 'cookies', 'error', 'content', 'support', 'skip', 'homepage', 'privacy', 'menu', 'site', 'sign', 'section', 'disabled', 'enabled', 'best', 'navigation', 'more', 'rights', 'supported', 'terms', 'signin', 'login', 'register', 'open', 'older', 'disable', 'failed', 'navigate', 'copyright', 'home', 'requested', 'help', 'know', 'jump', 'required', 'javascript', 'log', 'enable', 'reserved', 'policies', 'blocked', 'upgrade', 'anymore', 'copyrights', 'registered', 'condition', 'main', 'conditions', 'cookie', 'cookies', 'term'}

In [0]:
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')

def lemmatize(text):
  """ lemmatize by, taking in account they don't differ much from original words after it """

  lt = (WordNetLemmatizer().lemmatize(text))
  if lt not in dictionary: return text
  sm = model.similarity(text, lt)
  if sm >= 0.5 and sm <= 1: return lt 
  else: return text

def fine(s):
  """ Keep only string b/w length 3 and 45 as well as capitalize 3 char strings """

  if not s.isalpha(): return False
  if len(s) < 3 or len(s) > 45: return False
  if len(s) == 3 and s.islower(): return False
  return True


def mild_preprocess(content):
  """  1) tokenization
       2) remove stopwords and small words
       3) convert in lowercase """ 

  content = tokenize(content, deacc=True)
  content = list(filter(fine, content))
  content = [token.lower() for token in content]
  content = [lemmatize(token) for token in content if token not in STOPWORDS and token in dictionary and token not in AVOID]
  return content

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# SCRAPPING

In [0]:
!pip3 install selenium
!pip3 install tldextract
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 911kB 2.8MB/s 
     |████████████████████████████████| 51kB 1.7MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [91.7 kB]
Get:11 http://ppa.launchpad.net/marutte

In [0]:
from bs4 import BeautifulSoup
from bs4.element import Comment
from tldextract import extract
from selenium import webdriver
import requests
import warnings

warnings.filterwarnings("ignore")

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
browser.set_page_load_timeout(10)

headers = {
	'User-Agent': 'Mozilla/75.0',
	'Accept-Language': 'en-US,en;q=0.5'
}

SUFFICIENT = 70

In [0]:
def tag_visible(element):
  """ check visible tag """

  if element.parent.name in ['style', 'script', 'head', 'meta', '[document]']:
      return False
  if isinstance(element, Comment):
      return False
  return True

def text_from_html(html):
  """ Extract visible text from html body """

  soup = BeautifulSoup(html, 'html.parser')
  text = soup.findAll(text=True)
  visible_text = filter(tag_visible, text)  
  return u" ".join(t.strip() for t in visible_text)

In [0]:
def get_about_url(html, url):
  """  return url of about page if exist """
  try:
    domain = extract(url).domain.lower()  #fetch domain name of site
    soup = BeautifulSoup(html, 'html.parser')
  
    for link in soup.find_all('a'):
      if link.get('href') != None and 'about' in link.get('href').lower() and domain in link.get('href').lower():
        return link.get('href')
  except:
    return None


In [0]:
def get_static_text_content(url):
  """ scrap static content form url and preprocess it """
  content = []
  try:
    res = requests.get(url[1], headers=headers, verify=False, timeout=10)
    content.extend(mild_preprocess(text_from_html(res.text)))
    abt_url = get_about_url(res.text, url[1])
    if abt_url != None:
      res = requests.get(abt_url, headers=headers, verify=False, timeout=10)
      content.extend(mild_preprocess(text_from_html(res.text)))
    return [url[0], url[1], ' '.join(content)]
  except:
    return [url[0], url[1], ' '.join(content)]

In [0]:
def get_dynamic_text_content(url):
  """ scrap dynamic content form url and preprocess it """
  content = []
  try:
    browser.get(url)
    content.extend(mild_preprocess(text_from_html(browser.page_source)))
    abt_url = get_about_url(browser.page_source, url)
   
    if abt_url != None:
      browser.get(abt_url)
      content.extend(mild_preprocess(text_from_html(browser.page_source)))
    return content
  except:
    return content

def get_text_content(url):
  """ scrap text content from url """
  content = get_static_text_content(url[1])
  if (len(content) < SUFFICIENT): content = get_dynamic_text_content(url[1])
  return [url[0], url[1], ' '.join(content)]

In [0]:
#AVOID SCRAPPING DYNMAIC CONTENT WITH MULTIPROCESSING
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
import time

def scrap(i, j):
  with ProcessPoolExecutor(max_workers=50) as executor:
      start = time.time()
      futures = [ executor.submit(get_static_text_content, url) for url in domains[i:j]]
      results = []
      for result in tqdm(as_completed(futures)):
          results.append(result.result())
          with open("/content/drive/My Drive/Site Data/data_static_4.csv", 'a') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(result.result())
          
      end = time.time()
      print("\nTime Taken: {:.6f}s".format(end-start))

In [0]:
import csv
import sys

csv.field_size_limit(sys.maxsize)

with open("/content/drive/My Drive/Site Data/attempt.csv") as csvfile:
  csv_reader = csv.reader(csvfile)
  results = list(csv_reader)

attempt = result

from tqdm import tqdm

for url in tqdm(attempt[5400:]):
  result = get_text_content(url)
  with open("/content/drive/My Drive/Site Data/data_dynamic4.csv", 'a') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(result)